In [ ]:
import os
import pprint as pp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import csv
import glob
import hashlib
import sys
import struct
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing the dataset
from sklearn.model_selection import train_test_split
dataset = pd.read_csv('/content/drive/MyDrive/PfePhishing/4Phishing detection using Machine Learning techniques/dataset.csv')

In [ ]:
dataset.head()

,index,having_IPhaving_IP_Address,URLURL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,1,-1,1,1,1,-1,-1,-1,-1,-1,1,1,-1,1,-1,1,-1,-1,-1,0,1,1,1,1,-1,-1,-1,-1,1,1,-1,-1
1,2,1,1,1,1,1,-1,0,1,-1,1,1,-1,1,0,-1,-1,1,1,0,1,1,1,1,-1,-1,0,-1,1,1,1,-1
2,3,1,0,1,1,1,-1,-1,-1,-1,1,1,-1,1,0,-1,-1,-1,-1,0,1,1,1,1,1,-1,1,-1,1,0,-1,-1
3,4,1,0,1,1,1,-1,-1,-1,1,1,1,-1,-1,0,0,-1,1,1,0,1,1,1,1,-1,-1,1,-1,1,-1,1,-1
4,5,1,0,-1,1,1,-1,1,1,-1,1,1,1,1,0,0,-1,1,1,0,-1,1,-1,1,-1,-1,0,-1,1,1,1,1


In [ ]:
# Sepratating & assigning features and target columns to X & y
from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

y = dataset['Result']
X = dataset.drop('Result',axis=1)
X = X.fillna(0)


lb_make = LabelEncoder()
y = lb_make.fit_transform(y)
y


array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# Splitting the dataset into train and test sets: 80-20 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state = 12)
X_train.shape, X_test.shape

((8844, 31), (2211, 31))

# This section is for Performance results

In [ ]:
#importing packages
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [ ]:
# Creating holders to store the model performance results
ML_Model = []
acc_train = []
acc_test = []

#function to call for storing the results
def storeResults(model, a,b):
  ML_Model.append(model)
  acc_train.append(round(a, 3))
  acc_test.append(round(b, 3))

# Classifier: Decision Tree

In [ ]:
# Decision Tree model 
from sklearn.tree import DecisionTreeClassifier
import time
start_time = time.time()

# instantiate the model 
tree = DecisionTreeClassifier(max_depth = 10)
# fit the model 
tree.fit(X_train, y_train)
scores = cross_val_score(tree, X_train, y_train, cv=10)
scores.mean()
print("--- %s seconds ----" %(time.time() - start_time ))

--- 0.306870698928833 seconds ----


In [ ]:
#predicting the target value from the model for the samples
y_test_tree = tree.predict(X_test)
y_train_tree = tree.predict(X_train)

Performance Evaluation

In [ ]:
#computing the accuracy of the model performance
acc_train_tree = scores.mean()
acc_test_tree = accuracy_score(y_test,y_test_tree)

print("Decision Tree: Accuracy on training Data: {:.3f}".format(acc_train_tree))
print("Decision Tree: Accuracy on test Data: {:.3f}".format(acc_test_tree))

Decision Tree: Accuracy on training Data: 0.942
Decision Tree: Accuracy on test Data: 0.949


Storing the results

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('Decision Tree', acc_train_tree, acc_test_tree)

# Classifier: Random Forest

In [ ]:
# Random Forest model
from sklearn.ensemble import RandomForestClassifier
start_time = time.time()

# instantiate the model
forest = RandomForestClassifier(max_depth=10)

# fit the model 
forest.fit(X_train, y_train)
scores = cross_val_score(forest, X_train, y_train, cv=10)
scores.mean()
print("--- %s seconds ----" %(time.time() - start_time ))

--- 6.205198049545288 seconds ----


In [ ]:
#predicting the target value from the model for the samples
y_test_forest = forest.predict(X_test)
y_train_forest = forest.predict(X_train)

Performance Evaluation

In [ ]:
#computing the accuracy of the model performance
acc_train_forest = scores.mean()
acc_test_forest = accuracy_score(y_test,y_test_forest)

print("Random forest: Accuracy on training Data: {:.3f}".format(acc_train_forest))
print("Random forest: Accuracy on test Data: {:.3f}".format(acc_test_forest))

Random forest: Accuracy on training Data: 0.952
Random forest: Accuracy on test Data: 0.959


Storing the results

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('Random Forest', acc_train_forest, acc_test_forest)

# XGBoost Classifier 

In [ ]:
#XGBoost Classification model
from xgboost import XGBClassifier
start_time = time.time()

# instantiate the model
xgb = XGBClassifier(learning_rate=0.4,max_depth=10)
#fit the model
xgb.fit(X_train, y_train)
scores = cross_val_score(xgb, X_train, y_train, cv=10)
scores.mean()
print("--- %s seconds ----" %(time.time() - start_time ))

--- 25.92339849472046 seconds ----


In [ ]:
#predicting the target value from the model for the samples
y_test_xgb = xgb.predict(X_test)
y_train_xgb = xgb.predict(X_train)

Performance evaluation

In [ ]:
#computing the accuracy of the model performance
acc_train_xgb = scores.mean()
acc_test_xgb = accuracy_score(y_test,y_test_xgb)

print("XGBoost: Accuracy on training Data: {:.3f}".format(acc_train_xgb))
print("XGBoost : Accuracy on test Data: {:.3f}".format(acc_test_xgb))

XGBoost: Accuracy on training Data: 0.964
XGBoost : Accuracy on test Data: 0.966


Storing the results

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('XGBoost', acc_train_xgb, acc_test_xgb)

# **Classifier: Artificial Neural Network**

In [ ]:
#importing required packages
import keras
from keras.layers import Input, Dense
from keras import regularizers
import tensorflow as tf
from keras.models import Model
from sklearn import metrics

In [ ]:

from sklearn.preprocessing import StandardScaler                   # for normalization of our data
from keras.wrappers.scikit_learn import KerasClassifier            #package allowing keras to work with python
from sklearn.model_selection import cross_val_score, GridSearchCV  #using Kfold and if needed, GridSearch object in analysis
from sklearn.utils import shuffle                                  # shuffling our own made dataset
from keras.models import Sequential                                # linear layer stacks model for keras
from keras.layers import Dense, Dropout     

In [ ]:
    epoch = 20
        
    classifier = Sequential()
    classifier.add(Dense(units = 40, activation='relu',kernel_initializer='uniform', input_dim = 31))
    #classifier.add(Dropout(p= 0.1))
    classifier.add(Dense(units = 40, activation='relu',kernel_initializer='uniform'))
    #classifier.add(Dropout(p= 0.1))
    classifier.add(Dense(units = 40, activation='relu',kernel_initializer='uniform'))
    #classifier.add(Dropout(p= 0.1))
    classifier.add(Dense(units = 40, activation='relu',kernel_initializer='uniform'))      
    #classifier.add(Dropout(p= 0.1))   
    classifier.add(Dense(units = 1, activation='sigmoid',kernel_initializer='uniform'))
    #classifier.add(Dropout(p= 0.1))
    classifier.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    start_time = time.time()

    history = classifier.fit(X_train, y_train, epochs=epoch, validation_data=(X_test, y_test), verbose=1)
    print("--- %s seconds ----" %(time.time() - start_time ))
    

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Conv2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def visualisation_history(history):
  
  fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(13,5))

  ax1.plot(history.history['accuracy'],c='blue')
  ax1.plot(history.history['val_accuracy'],c='red')
  ax1.set_title('model accuracy')
  ax1.set_ylabel('accuracy')
  ax1.set_xlabel('epoch')
  ax1.legend(['train', 'test'], loc='center right')
  

  ax2.plot(history.history['loss'],c='blue')
  ax2.plot(history.history['val_loss'],c='red')
  ax2.set_title('model loss')
  ax2.set_ylabel('loss')
  ax2.set_xlabel('epoch')
  ax2.legend(['train', 'test'], loc='center right')
  plt.show()
# evaluation du modèle
_, acc =classifier.evaluate(X_test, y_test)
print('Test Accuracy: %.3f' % (acc * 100))
visualisation_history(history)

Performance Evaluation

In [ ]:
y_test_ann  = classifier.predict(X_test)
y_train_ann = classifier.predict(X_train)

In [ ]:
acc_train_ann = classifier.evaluate(X_train,y_train)[1]
acc_test_ann = classifier.evaluate(X_test,y_test)[1]

print('\nANN: Accuracy on training Data: {:.3f}' .format(acc_train_ann))
print('ANN: Accuracy on test Data: {:.3f}' .format(acc_test_ann))

70/70 [==============================] - 0s 1ms/step - loss: 0.1921 - accuracy: 0.9285

ANN: Accuracy on training Data: 0.923
ANN: Accuracy on test Data: 0.929


Storing the results

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('ANN', acc_train_ann, acc_test_ann)

# **Classifier: Convolutional Neural Network**


In [ ]:
from numpy import ndarray
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

AttributeError: ignored

In [ ]:
#importing required packages
import keras
from keras.layers import Input, Dense
from keras import regularizers
import tensorflow as tf
from keras.models import Model
from sklearn import metrics

In [ ]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.layers import Conv1D,MaxPool1D
from tensorflow.keras.optimizers import Adam

In [ ]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)


epoch = 20
model = Sequential()
model.add(Conv1D(32,2,activation='relu',input_shape = X_train[0].shape))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(64,2,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))

model.summary()

model.compile(optimizer=Adam(lr=0.0001), loss = 'binary_crossentropy', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 30, 32)            96        
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 32)            128       
_________________________________________________________________
dropout (Dropout)            (None, 30, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 29, 64)            4160      
_________________________________________________________________
batch_normalization_1 (Batch (None, 29, 64)            256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 29, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1856)             

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
# X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
epoch = 20
model = Sequential()
model.add(Conv1D(32,2,activation='relu',input_shape = X_train[0].shape))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(64,2,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))

#model.summary()

model.compile(optimizer=Adam(lr=0.0001), loss = 'binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
start_time = time.time()

history = model.fit(X_train, y_train, epochs=epoch, validation_data=(X_test, y_test), verbose=1)
print("--- %s seconds ----" %(time.time() - start_time ))
# classifier = KerasClassifier(build_fn= CNN_builder,
#                              batch_size = 10,
#                              nb_epoch = 100)


Epoch 1/20
277/277 [==============================] - 3s 10ms/step - loss: 0.1608 - accuracy: 0.9342 - val_loss: 0.1216 - val_accuracy: 0.9493
Epoch 2/20
277/277 [==============================] - 3s 11ms/step - loss: 0.1567 - accuracy: 0.9307 - val_loss: 0.1201 - val_accuracy: 0.9466
Epoch 3/20
277/277 [==============================] - 3s 10ms/step - loss: 0.1547 - accuracy: 0.9348 - val_loss: 0.1161 - val_accuracy: 0.9489
Epoch 4/20
277/277 [==============================] - 3s 11ms/step - loss: 0.1523 - accuracy: 0.9333 - val_loss: 0.1146 - val_accuracy: 0.9512
Epoch 5/20
277/277 [==============================] - 3s 10ms/step - loss: 0.1544 - accuracy: 0.9351 - val_loss: 0.1163 - val_accuracy: 0.9512
Epoch 6/20
277/277 [==============================] - 3s 10ms/step - loss: 0.1552 - accuracy: 0.9327 - val_loss: 0.1155 - val_accuracy: 0.9521
Epoch 7/20
277/277 [==============================] - 3s 10ms/step - loss: 0.1491 - accuracy: 0.9375 - val_loss: 0.1147 - val_accuracy: 0.9512

Performance evaluation

In [ ]:
acc_train_cnn = scores.mean()
acc_test_cnn = model.evaluate(X_test, y_test)[1]

print('\nCNN: Accuracy on training Data: {:.3f}' .format(acc_train_cnn))
print('CNN: Accuracy on test Data: {:.3f}' .format(acc_test_cnn))

Storing the results

In [ ]:
#storing the results. The below mentioned order of parameter passing is important.
#Caution: Execute only once to avoid duplications.
storeResults('CNN', acc_train_cnn, acc_test_cnn)

# Performance Comparison

In [ ]:
#creating dataframe
results = pd.DataFrame({ 'ML Model': ML_Model,    
    'Train Accuracy': acc_train,
    'Test Accuracy': acc_test})
results

,ML Model,Train Accuracy,Test Accuracy
0,Decision Tree,0.942,0.949
1,Random Forest,0.952,0.959
2,XGBoost,0.964,0.966
3,CNN,0.964,0.957


In [ ]:
#Sorting the datafram on accuracy
results.sort_values(by=['Test Accuracy', 'Train Accuracy'], ascending=False)

,ML Model,Train Accuracy,Test Accuracy
2,XGBoost,0.964,0.966
1,Random Forest,0.952,0.959
3,CNN,0.964,0.957
0,Decision Tree,0.942,0.949


In [ ]:
# # Save model
# model.save("CNN-Antiphishing")